## Imports

In [1]:
import cv2
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


## Local Phase Quantization (LPQ)

In [3]:

def lpq(img_filepath, winSize=3,freqestim=1,mode='nh'):

	img = cv2.imread(img_filepath,0)
	rho=0.90

	STFTalpha=1/winSize  # alpha in STFT approaches (for Gaussian derivative alpha=1)
	sigmaS=(winSize-1)/4 # Sigma for STFT Gaussian window (applied if freqestim==2)
	sigmaA=8/(winSize-1) # Sigma for Gaussian derivative quadrature filters (applied if freqestim==3)

	convmode='valid' # Compute descriptor responses only on part that have full neigborhood. Use 'same' if all pixels are included (extrapolates np.image with zeros).

	img=np.float64(img) # Convert np.image to double
	r=(winSize-1)/2 # Get radius from window size
	x=np.arange(-r,r+1)[np.newaxis] # Form spatial coordinates in window

	if freqestim==1:  #  STFT uniform window
	    #  Basic STFT filters
	    w0=np.ones_like(x)
	    w1=np.exp(-2*np.pi*x*STFTalpha*1j)
	    w2=np.conj(w1)

	## Run filters to compute the frequency response in the four points. Store np.real and np.imaginary parts separately
	# Run first filters
	filterResp1=convolve2d(convolve2d(img,w0.T,convmode),w1,convmode)
	filterResp2=convolve2d(convolve2d(img,w1.T,convmode),w0,convmode)
	filterResp3=convolve2d(convolve2d(img,w1.T,convmode),w1,convmode)
	filterResp4=convolve2d(convolve2d(img,w1.T,convmode),w2,convmode)

	# Initilize frequency domain matrix for four frequency coordinates (np.real and np.imaginary parts for each frequency).
	freqResp=np.dstack([filterResp1.real, filterResp1.imag,
                     
	                    filterResp2.real, filterResp2.imag,
	                    filterResp3.real, filterResp3.imag,
	                    filterResp4.real, filterResp4.imag])

	## Perform quantization and compute LPQ codewords
	inds = np.arange(freqResp.shape[2])[np.newaxis,np.newaxis,:]
	LPQdesc=((freqResp>0)*(2**inds)).sum(2)

	## Switch format to uint8 if LPQ code np.image is required as output
	if mode=='im':
		LPQdesc=np.uint8(LPQdesc)
		plt.figure(figsize=(20,10))
		plt.subplot(121)
		plt.imshow(img, cmap='gray')
		plt.title("Original image")

		plt.subplot(122)
		plt.imshow(LPQdesc, cmap='gray')
		plt.title("lpq")
		plt.show()


	## Histogram if needed
	if mode=='nh' or mode=='h':
	    LPQdesc=np.histogram(LPQdesc.flatten(),range(256))[0]

	## Normalize histogram if needed
	if mode=='nh':
	    LPQdesc=LPQdesc/LPQdesc.sum()

	return LPQdesc

## Train Images feature extraction

In [4]:
features = []
features.append(lpq("images/train/1-1.jpeg"))
features.append(lpq("images/train/1-2.jpeg"))
features.append(lpq("images/train/1-3.jpeg"))
features.append(lpq("images/train/2-1.jpeg"))
features.append(lpq("images/train/2-2.jpeg"))
features.append(lpq("images/train/2-3.jpeg"))
features.append(lpq("images/train/3-1.jpeg"))
features.append(lpq("images/train/3-2.jpeg"))
features.append(lpq("images/train/3-3.jpeg"))
features.append(lpq("images/train/4-1.jpeg"))
features.append(lpq("images/train/4-2.jpeg"))
features.append(lpq("images/train/4-3.jpeg"))
features = np.array(features)
classes = np.array([1,1,1,2,2,2,3,3,3,4,4,4])

## Fit classifier

#### KNN

In [59]:
knn = KNeighborsClassifier()
knn.fit(features, classes)

KNeighborsClassifier()

In [60]:
print(knn.predict([lpq("images/test/1.jpeg")]))
print(knn.predict([lpq("images/test/2.jpeg")]))
print(knn.predict([lpq("images/test/3.jpeg")]))
print(knn.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[3]
[3]


#### Decision Tree

In [6]:
dt = DecisionTreeClassifier()
dt.fit(features, classes)


[4]


In [9]:
print(dt.predict([lpq("images/test/1.jpeg")]))
print(dt.predict([lpq("images/test/2.jpeg")]))
print(dt.predict([lpq("images/test/3.jpeg")]))
print(dt.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[4]
[4]


#### SVM

In [41]:
clf = svm.SVC(kernel='rbf')
clf.fit(features, classes)

SVC()

In [42]:
print(clf.predict([lpq("images/test/1.jpeg")]))
print(clf.predict([lpq("images/test/2.jpeg")]))
print(clf.predict([lpq("images/test/3.jpeg")]))
print(clf.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[3]
[3]


#### Gaussian Bayes

In [56]:
gnb = GaussianNB()
bayes = gnb.fit(features, classes)

In [57]:
print(bayes.predict([lpq("images/test/1.jpeg")]))
print(bayes.predict([lpq("images/test/2.jpeg")]))
print(bayes.predict([lpq("images/test/3.jpeg")]))
print(bayes.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[3]
[3]


#### Minimum Distance Classifier

In [61]:
clf = NearestCentroid()
clf.fit(features, classes)

NearestCentroid()

In [62]:
print(clf.predict([lpq("images/test/1.jpeg")]))
print(clf.predict([lpq("images/test/2.jpeg")]))
print(clf.predict([lpq("images/test/3.jpeg")]))
print(clf.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[3]
[3]


#### Random Forest

In [82]:
clf = RandomForestClassifier(max_depth=5, random_state=4)
clf.fit(features, classes)

RandomForestClassifier(max_depth=5, random_state=4)

In [83]:
print(clf.predict([lpq("images/test/1.jpeg")]))
print(clf.predict([lpq("images/test/2.jpeg")]))
print(clf.predict([lpq("images/test/3.jpeg")]))
print(clf.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[3]
[4]


In [13]:
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(5, 2), random_state=1)
# clf = MLPClassifier(hidden_layer_sizes=(5,2),max_iter = 300,activation = 'relu', solver = 'adam',alpha=1e-5,)
clf.fit(features, classes)
print(clf.predict([lpq("images/test/1.jpeg")]))
print(clf.predict([lpq("images/test/2.jpeg")]))
print(clf.predict([lpq("images/test/3.jpeg")]))
print(clf.predict([lpq("images/test/4.jpeg")]))

[1]
[1]
[3]
[3]
